In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from astropy.time import Time
from scipy.interpolate import interp1d
import math
import copy
import glob
import subprocess
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
#Some routines

# Routine to get the date of the first valid row of each file
def get_date_from_meteo_file(filename):
    df = pd.read_csv(filename, header=1, parse_dates={'Date':['YYYY/MM/JJ',' hh:mn:sec(HL)']})
    # date of first valid row
    dd = df['Date'][0].date()
    return (dd)

# Read all files for a given instrument, resample and concatenate
def read_single_meteo_file(filename,keep=['Date', 'pressure', 'Temperature', 'Humidity', 'windSpeed', 'windDir'],set_index=True):
    
    count = len(open(filename).readlines(  ))
    if count > 4:
        df = pd.read_csv(filename, header=1, parse_dates={'Date':['YYYY/MM/JJ',' hh:mn:sec(HL)']})
        # Remove spaces in column names
        df.rename(str.strip,axis='columns',inplace=True)
        # Rename some columns
        df.rename(columns={"outTemp": "Temperature", "outHumidity": "Humidity"},inplace=True)
        if (keep != 'all'):
            columns=keep
            index_to_remove = df.columns.drop(keep)
            df = df.drop(columns=index_to_remove)
        if (set_index):
            # Use Date as index
            df.set_index('Date',inplace=True)
            
        
        df['windSpeed'] = df['windSpeed']/(3.6)  #km/h to m/s
       
        
                        
        # replace no valid value with nan for wind direction
        tonan(df['windDir'])
        #tonan(p_cats)
        #tonan(temp_cats)
        #tonan(hu_cats)
        
        
        # extract the index of nan value from wind direction
        df.loc[pd.isna(df["windDir"]), :].index
        
        # replace alsa nan for same index in windspeed (cz by default they are replaced with 0)
        #df.loc[df.loc[pd.isna(df["windDir"]), :].index,'windSpeed'] = np.nan
        
        df['windDir'][df['windDir'] > 300] = df['windDir'][df['windDir'] > 300] - 360       
    else:
        df = pd.DataFrame()
    return (df)


# Routine to prepare training samples from single contiguous dataframe
def split_series(series, n_past, n_future, out_column_index=5):
    #
    # n_past ==> no of past observations
    #
    # n_future ==> no of future observations 
    #
    out_column_index
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        # slicing the past and future parts of the window
        past, future = series[window_start:past_end, :], series[past_end:future_end, [out_column_index]]
        X.append(past)
        y.append(future)
    return np.array(X), np.array(y)  


# Routine to fill missing values with random values from data distribution
def fill_missing_values(dataframe,random=False):

    if (random):
        columns = dataframe.columns
        for col in columns:
            mean = dataframe[col].mean()
            std  = dataframe[col].std() 
            mask_nans = dataframe[col].isnull().values
            randoms = np.random.standard_normal(mask_nans.sum())*std + mean
            dataframe.loc[mask_nans,col]=randoms
    else:
        dataframe.interpolate(method='spline',order=3,inplace=True)
    return(dataframe)

# Routine to change none valid value to nan
def tonan(x):
    x[x==-9999] = np.nan

In [3]:
def simple_data_preparation(data_dir = '/Users/cgiordano/Documents/Travail/WRF/Calern_ML/Data', 
                     meteo_dir='CATS',meteo_tag='meteo_cats_*.csv',
                     sampling_rate_min=5,interpolate=True):
    

    sampling_rate_str = '%dmin'%sampling_rate_min

   
    METEO_files = np.array(glob.glob(data_dir+'/'+meteo_dir+'/'+meteo_tag))
    filetest = np.array(['-'.join(item.split('_')[-1].split('.')[0].split('-')[::-1]) for item in METEO_files])
    idx = np.argsort(filetest)
    METEO_files = [item for item in METEO_files[idx]]
    nMETEO = len(METEO_files)


    framelist = []
    for file in METEO_files:
        print("Processing file %s"%file)
        df = read_single_meteo_file(file)
        if (df.empty):
            continue
        df = df.resample(sampling_rate_str).bfill()
        if (interpolate):
            df.interpolate('spline',order=1,inplace=True)
        framelist.append(df)
    meteo_data = pd.concat(framelist)
    
    
    # Add group index, based on time jumps bigger than 300s (this needs to be in sync with sampling_rate !!)
    meteo_data['groups'] = (meteo_data.index.to_series().diff().dt.seconds > 300).cumsum()
    return meteo_data

In [4]:
meteo_data = simple_data_preparation(data_dir = r'C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1', 
                     meteo_dir='CATS',meteo_tag='meteo_cats_*.csv',
                     sampling_rate_min=5,interpolate=True)

Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_01-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_02-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_03-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_04-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_05-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_06-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_07-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_08-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_09-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_10-01-2015.csv
Processing

C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x==-9999] = np.nan
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['windDir'][df['windDir'] > 300] = df['windDir'][df['windDir'] > 300] - 360
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_16-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_17-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_18-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_19-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_20-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_21-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_22-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_23-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_24-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_25-01-2015.csv
Processing

C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x==-9999] = np.nan
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['windDir'][df['windDir'] > 300] = df['windDir'][df['windDir'] > 300] - 360
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_31-01-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_01-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_02-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_03-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_04-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_05-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_06-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_07-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_08-02-2015.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_09-02-2015.csv
Processing

...

Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_30-10-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_04-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_05-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_06-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_07-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_08-11-2020.csv


C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x==-9999] = np.nan
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['windDir'][df['windDir'] > 300] = df['windDir'][df['windDir'] > 300] - 360
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_09-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_10-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_11-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_12-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_13-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_14-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_15-11-2020.csv


C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x==-9999] = np.nan
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['windDir'][df['windDir'] > 300] = df['windDir'][df['windDir'] > 300] - 360
C:\Users\Mary-\AppData\Local\Temp\ipykernel_12264\3560210474.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_16-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_17-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_18-11-2020.csv
Processing file C:\Users\Mary-\Desktop\PhD\First year\ML LSTM\test1/CATS\meteo_cats_19-11-2020.csv


In [5]:
#take time into consideration
meteo_data['Date'] = meteo_data.index
Date_time = meteo_data['Date']
timestamp_s = Date_time.map(pd.Timestamp.timestamp)

day = 24*60*60
year = (365.2425)*day

meteo_data['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
meteo_data['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
meteo_data['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
meteo_data['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

meteo_data = meteo_data.drop(['Date'], axis=1)
meteo_data

,pressure,Temperature,Humidity,windSpeed,windDir,groups,Day sin,Day cos,Year sin,Year cos
Date,,,,,,,,,,
2015-01-01 00:00:00,1021.766370,10.722222,47.0,0.000000,NaN,0,1.232357e-12,1.000000,0.001505,0.999999
2015-01-01 00:05:00,1021.631269,10.722222,47.0,0.000000,NaN,0,2.181489e-02,0.999762,0.001565,0.999999
2015-01-01 00:10:00,1021.631269,10.722222,47.0,0.000000,NaN,0,4.361939e-02,0.999048,0.001625,0.999999
2015-01-01 00:15:00,1021.631269,10.722222,47.0,0.000000,NaN,0,6.540313e-02,0.997859,0.001684,0.999999
2015-01-01 00:20:00,1021.631269,10.722222,47.0,0.000000,NaN,0,8.715574e-02,0.996195,0.001744,0.999998
...,...,...,...,...,...,...,...,...,...,...
2020-11-19 00:00:00,884.260574,11.188889,29.0,3.460868,298.851129,63,-1.674314e-12,1.000000,-0.666004,0.745948
2020-11-19 00:05:00,884.252894,11.244444,28.0,3.222925,291.321231,63,2.181489e-02,0.999762,-0.665959,0.745988
2020-11-19 00:10:00,884.481590,11.411111,28.0,2.110863,281.611996,63,4.361939e-02,0.999048,-0.665915,0.746028


In [6]:
def prepare_learning_sets(dataframe,input_sequence_length_min=60,
                          output_sequence_length_min=30,test_size=0.2,
                          sampling_rate_min=5, 
                          out_column='Temperature',
                          return_scaler = True):

    '''
    This routine first splits the dataframe into training and testing, scales all columns, and finally arrange the
    training and testing sets into their final form for training and testing. The final form consists in an array 
    of time series of input_sequence_length_min minutes and corresponding target time series of 
    output_sequence_length_min minutes. 
    '''
    n_input_sequence = input_sequence_length_min//sampling_rate_min
    n_output_sequence = output_sequence_length_min//sampling_rate_min
    n_full_sequence  = n_input_sequence+n_output_sequence
    n_features = dataframe.shape[1]
    X_train = np.empty((0,n_input_sequence,n_features))
    X_test = np.empty((0,n_input_sequence,n_features))
    Y_train = np.empty((0,n_output_sequence,1))
    Y_test = np.empty((0,n_output_sequence,1))
    out_column_index = dataframe.columns.get_loc(out_column)

    for group in np.unique(dataframe['groups']):
        current = dataframe[dataframe.groups==group]
        

        # Split and check that current dataframes have enough samples
        train_df, test_df = train_test_split(current,test_size=test_size,shuffle=False)
     

        if (train_df.shape[0] < n_full_sequence or test_df.shape[0] < n_full_sequence):
            continue

        
        for col in train_df.columns:
            scaler = MinMaxScaler(feature_range=(-1,1))
            ss = scaler.fit_transform(train_df[col].values.reshape((-1,1)))
            ss = ss.reshape(len(ss)) # Get rid of extra dim
            train_df.loc[:,col] = ss
            train_df.rename(columns={col:"scaled_%s"%col},inplace=True)
        
            if col==out_column:
                out_scaler = scaler
            # Now process the corresponding column from the test dataframe
            ss = scaler.transform(test_df[col].values.reshape((-1,1)))
            ss = ss.reshape(len(ss))
            test_df.loc[:,col] = ss
            test_df.rename(columns={col:"scaled_%s"%col},inplace=True)


        # Fill missing values
        train_df = fill_missing_values(train_df)
        test_df  = fill_missing_values(test_df)
        

        # Now chunk the sets
        x_train, y_train = split_series(train_df.values,n_input_sequence,n_output_sequence,out_column_index=out_column_index)
        x_test,  y_test  = split_series(test_df.values, n_input_sequence,n_output_sequence,out_column_index=out_column_index)
        
        print('x_train',x_train.shape)
        print('y_train',y_train.shape)
        print('x_test',x_test.shape)
        print('y_test',y_test.shape)
        
        
        X_train = np.append(X_train,x_train,axis=0)
        Y_train = np.append(Y_train,y_train,axis=0)
        X_test  = np.append(X_test,x_test,axis=0)
        Y_test  = np.append(Y_test,y_test,axis=0)
    if (return_scaler):
        return(X_train,Y_train,X_test,Y_test,out_scaler)
    else:
        return (X_train,Y_train,X_test,Y_test)

In [7]:
X_train, Y_train, X_test, Y_test,out_scaler =  prepare_learning_sets(meteo_data,input_sequence_length_min=60,
                          output_sequence_length_min=30,test_size=0.2,
                          sampling_rate_min=5, 
                          out_column='Temperature',
                          return_scaler = True)

C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anacond

x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1

C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anacond

x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1

C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\Mary-\anaconda3\envs\ML\lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (60, 12, 10)
y_train (60, 6, 1)
x_test (3, 12, 10)
y_test (3, 6, 1)
x_train (60, 12, 10)
y_train (60, 6, 1)
x_test (3, 12, 10)
y_test (3, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (209, 12, 10)
y_train (209, 6, 1)
x_test (40, 12, 10)
y_test (40, 6, 1)
x_train (103, 12, 10)
y_train (103, 6, 1)
x_test (14, 12, 10)
y_test (14, 6, 1)
x_train (233275, 12, 10)
y_train (233275, 6, 1)
x_test (58307, 12, 10)
y_test (58307, 6, 1)
x_train (18567, 12, 10)
y_train (18567, 6, 1)
x_test (4630, 12, 10)
y_test (4630, 6, 1)
x_train (13622, 12, 10)
y_tr